<a href="https://colab.research.google.com/github/evelyntr/Cal-Poly-301-lab-6-phase-1/blob/main/Copy_of_DATA_301_Lab_6B_Evelyn_Tran.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Bing Maps API

In this part of the lab, you will join the CityBikes data from the previous part of the lab with additional data that you will query from the Bing Maps API.

First, you will need to register for a Bing Maps Key. Follow the instructions [here](https://docs.microsoft.com/en-us/bingmaps/getting-started/bing-maps-dev-center-help/getting-a-bing-maps-key). You should be able to sign in with your Cal Poly account. When you get to Step 4, select:

- Key type: Basic
- Application type: Dev/Test

You will be able to make 125000 free requests with the resulting API key. This should be more than enough to complete this assignment.

We will be working with the [REST services in the Bing Maps API](https://docs.microsoft.com/en-us/bingmaps/rest-services/). Click on the link for a complete documentation of the features.

In [375]:
import pandas as pd
from google.colab import drive

drive.mount('/content/drive')
path = '/content/drive/My Drive/citybikes_us.csv'

df = pd.read_csv(path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Question 1

Read in the `DataFrame` of bike stations in the United States from Part A of this lab. Restrict to the stations in the "Bay Wheels" network (with network ID "ford-gobike").

How many of these stations are in the city/county of San Francisco? 

(_Hint:_ Use the [Locations API](https://docs.microsoft.com/en-us/bingmaps/rest-services/locations/) to get the address associated with each latitude and longitude coordinate.)

In [376]:
import requests 
id = "ford-gobike"
resp = requests.get("http://api.citybik.es/v2/networks/%s?filter=stations" % id)
data = resp.json()


In [377]:
destinations = []
station_names = []

In [378]:
BingMapsKey = "Av8fZK0SkX9GOtUInfDQ1soe6l6Ou5jlN1P_m4TYKNkp6y5fAvyOJ0w946ZyJlyd"
num_stations = 0
for station in data["network"]["stations"]:
  x = str(station["latitude"]) 
  y = str(station["longitude"])
  point = x + "," + y

  resp = requests.get("http://dev.virtualearth.net/REST/v1/Locations/%s?o=json&key=%s" % (point, BingMapsKey))
  data = resp.json()
  res = data["resourceSets"]
  if res != None:
    if (len(res) != 0):
      addr = data["resourceSets"][0]['resources'][0]['address']['formattedAddress']
      if ("San Francisco" in addr):
        num_stations += 1
        entry = {}
        entry['latitude'] = data["resourceSets"][0]['resources'][0]['point']['coordinates'][0]
        entry['longitude'] = data["resourceSets"][0]['resources'][0]['point']['coordinates'][1]
        destinations.append(entry)
        station_names.append(station['name'])

print("The number of stations in SF is:", num_stations)

#   print(resp.json())
# x = str(data["network"]["stations"][0]["latitude"])
# y = str(data["network"]["stations"][0]["longitude"])
# x
# y
# point = x + ','+ y
# point
# resp = requests.get("http://dev.virtualearth.net/REST/v1/Locations/%s?o=json&key=%s" % (point, BingMapsKey))
# data = resp.json()

# addr = data["resourceSets"][0]['resources'][0]
# addr['point']['coordinates']

The number of stations in SF is: 99


## Question 2

You want to go to Coit Tower. To save money, you decide to ride a "Bay Wheels" bike to the closest station and hail a cab from there to Coit Tower. What station should you bike to so that you are as close to Coit Tower as possible (as measured by driving distance)? Does your answer agree with the one that you obtained in Part A of this lab? If not, why does it differ?

_Hints:_ 
- You should restrict your attention to bike stations that are in San Francisco, which you determined in Question 1. 
- Use the [Routes API](https://docs.microsoft.com/en-us/bingmaps/rest-services/routes/calculate-a-distance-matrix) to calculate a distance matrix between Coit Tower and the bike stations. 
- You can do this with just one call to the API. Because there are too many stations, it is impossible to specify all the locations in the URL. Instead, you should make a POST request (`requests.post`), passing in the parameters as a JSON object through the `json=` parameter of `requests.post`. Read the API documentation carefully to learn how to use the POST API.

In [379]:
import requests

url = "https://dev.virtualearth.net/REST/v1/Routes/DistanceMatrix?key=Av8fZK0SkX9GOtUInfDQ1soe6l6Ou5jlN1P_m4TYKNkp6y5fAvyOJ0w946ZyJlyd"

origins = [{"latitude": 37.802747,"longitude": -122.405861}]

destinations

header = {
    'Content-Type': "application/json"
}

body = {
    "origins":origins, 
    "destinations": destinations, 
    "travelMode": "driving"
    }

resp = requests.post(url, headers=header, json=body)
data = resp.json()

dist_matrix = data["resourceSets"][0]['resources'][0]['results']
sorted_matrix = sorted(dist_matrix, key=lambda x: x["travelDistance"])
sorted_matrix
index_min_dist = sorted_matrix[0]['destinationIndex']
destinations[index_min_dist]
station_names[index_min_dist]


'North Point St at Powell St'

## Submission Instructions

- Copy this notebook to your own Drive, if you have not already.
- Restart this notebook and run the cells from beginning to end. 
  - Go to Runtime > Restart and Run All.
- Rename this notebook by clicking on "DATA 301 Lab 6B - YOUR NAMES HERE" at the very top of this page. Replace "YOUR NAMES HERE" with the first and last names of you (and your partners, for Phase 2).
- Get the link to your notebook:
  - Click on "Share" at the top-right. 
  - Change the settings to "Anyone with the link can view". 
  - Copy the sharing link into Canvas.